In [272]:
import pandas as pd
from keras.src.layers import BatchNormalization, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Input, Conv1D, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.model_selection import train_test_split

In [273]:
tf.random.set_seed(42)

In [274]:
heartbeats_df = pd.read_csv("../files/mitbih_test.csv", sep=',', header=None)

In [275]:
heartbeats_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,1.000000,0.758264,0.111570,0.000000,0.080579,0.078512,0.066116,0.049587,0.047521,0.035124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.908425,0.783883,0.531136,0.362637,0.366300,0.344322,0.333333,0.307692,0.296703,0.300366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.730088,0.212389,0.000000,0.119469,0.101770,0.101770,0.110619,0.123894,0.115044,0.132743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.000000,0.910417,0.681250,0.472917,0.229167,0.068750,0.000000,0.004167,0.014583,0.054167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.570470,0.399329,0.238255,0.147651,0.000000,0.003356,0.040268,0.080537,0.070470,0.090604,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.000000,0.923664,0.656489,0.195929,0.111959,0.175573,0.122137,0.050891,0.035623,0.055980,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.000000,0.797260,0.320548,0.043836,0.049315,0.065753,0.030137,0.008219,0.005479,0.010959,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.909408,0.975610,0.533101,0.134146,0.066202,0.000000,0.010453,0.012195,0.031359,0.146341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.927803,0.866359,0.299539,0.000000,0.231951,0.317972,0.274962,0.262673,0.270353,0.268817,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.000000,0.914230,0.473684,0.000000,0.064327,0.317739,0.405458,0.391813,0.382066,0.401559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [276]:
heartbeats_df.shape

(21892, 188)

In [277]:
heartbeats_df.describe()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
count,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,...,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000,21892.000000
mean,0.894410,0.761902,0.426627,0.221596,0.201676,0.209891,0.204805,0.200992,0.197634,0.196022,...,0.004588,0.004327,0.004020,0.003789,0.003638,0.003459,0.003166,0.003000,0.002946,0.473689
std,0.234560,0.218659,0.228572,0.208711,0.177727,0.172194,0.177946,0.176142,0.170228,0.166707,...,0.043128,0.042187,0.040255,0.039397,0.038535,0.037717,0.035903,0.035522,0.035266,1.143447
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.924260,0.683366,0.251197,0.050505,0.082873,0.087912,0.072663,0.065997,0.064516,0.068493,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.990431,0.828996,0.432777,0.167630,0.147642,0.158111,0.144068,0.144509,0.150422,0.149029,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.912319,0.583991,0.347092,0.259211,0.287356,0.298453,0.294563,0.289907,0.282956,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.991429,...,0.980392,1.000000,0.966102,1.000000,1.000000,1.000000,1.000000,0.996053,1.000000,4.000000


In [278]:
heartbeats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21892 entries, 0 to 21891
Columns: 188 entries, 0 to 187
dtypes: float64(188)
memory usage: 31.4 MB


In [279]:
y = heartbeats_df.iloc[:, -1]
col_name = heartbeats_df.columns[-1]
heartbeats_df = heartbeats_df.drop(col_name, axis=1)

In [280]:
y.unique()

array([0., 1., 2., 3., 4.])

In [281]:
scalar = MinMaxScaler()
heartbeats_df = scalar.fit_transform(heartbeats_df)

In [282]:
y = to_categorical(y, 5)
X = heartbeats_df.reshape(X.shape[0], X.shape[1], 1)

In [283]:
print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  (21892, 187, 1)
y shape:  (21892, 5)


In [284]:
param_grid = {
    'filters': [64, 128],
    'kernel_size': [5, 9],
    'dropout': [0.2, 0.3],
    'dense_units': [32, 64]
}

results = []

for filters in param_grid['filters']:
    for kernel_size in param_grid['kernel_size']:
        for dropout in param_grid['dropout']:
            for dense_units in param_grid['dense_units']:
                print(f"\n=== filters={filters}, kernel_size={kernel_size}, dropout={dropout}, dense_units={dense_units} ===")

                model = Sequential([
                    Input(shape=(X_train.shape[1], 1)),
                    Conv1D(filters, kernel_size, activation='relu', padding='same'),
                    BatchNormalization(),
                    Dropout(dropout),
                    Conv1D(filters, kernel_size, activation='relu', padding='same'),
                    BatchNormalization(),
                    Dropout(dropout),
                    GlobalMaxPooling1D(),
                    Dense(dense_units, activation='relu'),
                    Dropout(dropout),
                    Dense(5, activation='softmax')
                ])

                model.compile(
                    optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy']
                )

                early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
                reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

                history = model.fit(
                    X_train, y_train,
                    epochs=10,
                    batch_size=64,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stop, reduce_lr],
                    class_weight=class_weight_dict,
                    verbose=0  # чтобы не засорять вывод
                )

                val_loss, val_acc = model.evaluate(X_test, y_test, verbose=0)
                print(f"Validation accuracy: {val_acc:.4f}")
                results.append({
                    'filters': filters,
                    'kernel_size': kernel_size,
                    'dropout': dropout,
                    'dense_units': dense_units,
                    'val_acc': val_acc
                })


results = sorted(results, key=lambda x: x['val_acc'], reverse=True)
print("\nЛучшие комбинации параметров:")
for r in results[:5]:
    print(r)


In [285]:
model = Sequential([
    Input(shape=(X.shape[1], 1)),
    Conv1D(128, 9, activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.3),
    Conv1D(128, 5, activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.3),
    Conv1D(256, 5, activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.2),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')
])

In [286]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', 'categorical_accuracy']
)

In [287]:
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

In [288]:
y_labels = np.argmax(y, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_labels), y=y_labels)
class_weight_dict = dict(enumerate(class_weights))

In [289]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [290]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stop, reduce_lr],
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.3301 - categorical_accuracy: 0.3301 - loss: 2.3738 - val_accuracy: 0.1210 - val_categorical_accuracy: 0.1210 - val_loss: 1.4800 - learning_rate: 0.0010
Epoch 2/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.4019 - categorical_accuracy: 0.4019 - loss: 1.1386 - val_accuracy: 0.6186 - val_categorical_accuracy: 0.6186 - val_loss: 1.0774 - learning_rate: 0.0010
Epoch 3/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4372 - categorical_accuracy: 0.4372 - loss: 0.9123 - val_accuracy: 0.5188 - val_categorical_accuracy: 0.5188 - val_loss: 0.9919 - learning_rate: 0.0010
Epoch 4/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4967 - categorical_accuracy: 0.4967 - loss: 0.8517 - val_accuracy: 0.6150 - val_categorical_accuracy: 0.6150 - val_loss: 0.7806 - learning_rate: 0.0010
Epoch 5/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.5111 - categorical_accuracy: 0.5111 - loss: 0.8157 - val_

In [291]:
loss, acc, cat_acc = model.evaluate(X, y)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f"Categorical accuracy: {cat_acc:.4f}")

685/685 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8305 - categorical_accuracy: 0.8305 - loss: 0.5953
Loss: 0.5622
Accuracy: 0.8352
Categorical accuracy: 0.8352
